# Home energy management with linear peak demand tariff

In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import datetime
import pandas as pd
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

from utils import *

# import matplotlib as mpl
# import tqdm
# import matplotlib.ticker as ticker

save_figs = False
run_sensitivity_analysis = False

## Prescient optimization

### No storage

In [2]:
# Load and price data
load_data = pd.read_pickle("data/load_data.pickle")
price_data = pd.read_pickle("data/price_data.pickle")

# Forecast models
baseline = pd.read_pickle("data/load_baseline.pickle")
autoreg_residual_params = pd.read_pickle("data/autoreg_residual_params.pickle")

# Simulation window (1 year with hourly resolution)
T = 24 * 365
start_date = pd.Timestamp("2022-01-01 00:00:00")
sim_start_time = load_data.index.get_loc(start_date)
sim_end_time = sim_start_time + T
sim_index = load_data.index[sim_start_time:sim_end_time]
month_slices = get_month_slices(load_data[sim_start_time:sim_end_time].index)

FileNotFoundError: [Errno 2] No such file or directory: 'data/autoreg_residual_params.pickle'

In [ ]:
# Problem data (constants and parameters)
p_load = load_data.iloc[sim_start_time:sim_end_time].values
Q = cp.Parameter(nonneg=True, value=0)
C = cp.Parameter(nonneg=True, value=Q.value / 2)
D = cp.Parameter(nonneg=True, value=Q.value / 2)
q_init = cp.Parameter(nonneg=True, value=Q.value / 2)
q_final = cp.Parameter(nonneg=True, value=Q.value / 2)
alpha = 0.01
eff_charge = 0.95
eff_discharge = 0.95
E = 20
price_buy = price_data.iloc[sim_start_time:sim_end_time].values
price_peak = 252

# Create variables
p_grid = cp.Variable(T)
p_batt_in = cp.Variable(T)
p_batt_out = cp.Variable(T)
q = cp.Variable(T)

# Constraints
cons = [q[1:] == (1 - alpha) * q[:-1] + eff_charge * p_batt_in[1:] 
        - (1/eff_discharge) * p_batt_out[1:],
        q[0] == (1 - alpha) * q_init + eff_charge * p_batt_in[0] 
        - (1/eff_discharge) * p_batt_out[0],
        q[-1] == q_final, q <= Q, q >= 0,
        p_batt_in >= 0, p_batt_in <= C, p_batt_out >= 0, p_batt_out <= D,
        p_grid >= 0, p_grid <= E,
        p_grid + p_batt_out == p_batt_in + p_load]

# Create objective
cost_usage = cp.sum(cp.multiply(price_buy, p_grid))
cost_peak = cp.sum([price_peak * cp.max(p_grid[month]) for month in month_slices])
cost = cost_usage + cost_peak

# Create problem and solve
prescient = cp.Problem(cp.Minimize(cost), cons)
prescient.solve()

# Report results
print_costs(cost_usage.value, cost_peak.value)
plot_power_grid(sim_index, p_grid.value, Q.value)
ymin, ymax = plt.gca().get_ylim()
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

if save_figs:
    plt.savefig("figures/prescient_linear_no_storage.pdf")

### Storage capacity: 5 kWh

In [ ]:
# Update Q and related values
Q.value = 5
C.value = Q.value / 2
D.value = Q.value / 2
q_init.value = Q.value / 2
q_final.value = Q.value / 2

# Resolve the problem
prescient.solve()

# Report results
print_costs(cost_usage.value, cost_peak.value)
plot_power_grid(sim_index, p_grid.value, Q.value)
ymin, ymax = plt.gca().get_ylim()
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

plot_state_of_charge(sim_index, q.value)
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

if save_figs:
    plt.savefig("figures/prescient_linear_5kWh.pdf")


### Storage capacity: 15 kWh

In [ ]:
# Update Q and related values
Q.value = 15
C.value = Q.value / 2
D.value = Q.value / 2
q_init.value = Q.value / 2
q_final.value = Q.value / 2

# Resolve the problem
prescient.solve()

# Compute cost
cost_usage_val = np.sum(price_data[sim_index] * p_grid.value)
power_series = pd.Series(p_grid.value, index=sim_index)
peak_series = power_series.resample('M').max().values
cost_peak_val = np.sum(peak_series * price_peak)
prescient_cost = cost_usage_val + cost_peak_val

# Report results
# prescient_cost = cost_usage.value + cost_peak.value
print_costs(cost_usage.value, cost_peak.value)
plot_power_grid(sim_index, p_grid.value, Q.value)
ymin, ymax = plt.gca().get_ylim()
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

plot_state_of_charge(sim_index, q.value)
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

if save_figs:
    plt.savefig("figures/prescient_linear_15kWh.pdf")

### Storage capacity: 30 kWh

In [ ]:
# Update Q and related values
Q.value = 30
C.value = Q.value / 2
D.value = Q.value / 2
q_init.value = Q.value / 2
q_final.value = Q.value / 2

# Resolve the problem
prescient.solve()

# Report results
print_costs(cost_usage.value, cost_peak.value)
plot_power_grid(sim_index, p_grid.value, Q.value)
ymin, ymax = plt.gca().get_ylim()
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

plot_state_of_charge(sim_index, q.value)
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

if save_figs:
    plt.savefig("figures/prescient_linear_30kWh.pdf")

### Cost versus storage capacity

In [ ]:
if run_sensitivity_analysis:
    # Define the storage capacity range
    storage_capacities = np.linspace(0, 50)

    # Initialize arrays for storing costs and cost breakdown
    cost_usage_values = np.zeros(len(storage_capacities))
    cost_peak_values = np.zeros(len(storage_capacities))

    # Run simulations for various storage capacities
    for i, capacity in tqdm.tqdm(enumerate(storage_capacities), total=len(storage_capacities), desc="Performing sensitivity analysis"):
        # Update Q and related values
        Q.value = capacity
        C.value = capacity / 2
        D.value = capacity / 2
        q_init.value = capacity / 2
        q_final.value = Q.value / 2
        # Resolve the problem
        prescient.solve()
        # Save results
        cost_usage_values[i], cost_peak_values[i] = cost_usage.value, cost_peak.value
        
    # Plot the results: Total cost vs storage capacity
    fig, ax = plt.subplots()
    ax.plot(storage_capacities, (cost_usage_values + cost_peak_values)/1e3, marker='o', linestyle='-', markerfacecolor='blue', color='blue', markeredgewidth=1, markeredgecolor='black', label="Total cost")
    ax.plot(storage_capacities, cost_peak_values/1e3, marker='o', linestyle='-', markerfacecolor='green', color='green', markeredgewidth=1, markeredgecolor='black', label="Peak demand cost")
    ax.plot(storage_capacities, cost_usage_values/1e3, marker='o', linestyle='-', markerfacecolor='red', color='red', markeredgewidth=1, markeredgecolor='black', label="Energy cost")

    ax.set_xlabel("Storage capacity (kWh)")
    ax.set_ylabel("Cost (thousands of NOK)")
    ax.legend()
    ax.grid(True)
    plt.tight_layout()
    ymin, ymax = plt.gca().get_ylim()
    plt.gca().set_ylim(0, ymax);

    if save_figs:
        plt.savefig("figures/cost_vs_storage_capacity.pdf")

## Model predictive control

### Seasonal baseline forecast 

In [ ]:
# Constants
Q = 15
C, D, q_final = Q/2, Q/2, Q/2

def solve_problem(T_MPC, p_load, price_buy, prev_peak, q_init, month_slices):
    # Create variables
    p_grid = cp.Variable(T_MPC)
    p_batt_in = cp.Variable(T_MPC)
    p_batt_out = cp.Variable(T_MPC)
    q = cp.Variable(T_MPC)
    
    # Constraints
    cons = [q[0] == (1 - alpha) * q_init + eff_charge * p_batt_in[0] 
            - (1/eff_discharge) * p_batt_out[0],
            q[-1] == q_final, q <= Q, q >= 0,
            p_batt_in >= 0, p_batt_in <= C, p_batt_out >= 0, p_batt_out <= D,
            p_grid >= 0, p_grid <= E,
            p_grid + p_batt_out == p_batt_in + p_load]
    
    if T_MPC > 1:
            cons.extend([q[1:] == (1 - alpha) * q[:-1] + eff_charge * p_batt_in[1:] 
            - (1/eff_discharge) * p_batt_out[1:]])
    
    cost_usage = cp.sum(cp.multiply(price_buy, p_grid))
    previous_peaks = [prev_peak] + [0] * (len(month_slices) - 1)
#     cost_peak = cp.sum([price_peak * cp.pos(cp.max(p_grid[month]) - previous_peak) for month, previous_peak in zip(month_slices, previous_peaks)])
    cost_peak = cp.sum([price_peak * cp.pos(cp.max(p_grid[month]) - prev_peak_val) for month, prev_peak_val in zip(month_slices, previous_peaks)])
    cost = cost_usage + cost_peak
    
    # Create problem and solve
    mpc = cp.Problem(cp.Minimize(cost), cons)
    mpc.solve()
    
    return q.value[0], p_grid.value[0]

In [ ]:
# MPC prediction horizon
T_MPC = 24 * 30

# Initial conditions
q_init = Q/2
prev_peak = 0
    
# Empty arrays for storing results
energy_stored = np.empty(T)
power = np.empty(T)
peak = np.empty(T)

def make_forecasts(t, T_MPC):
    p_load = baseline[sim_start_time+t:sim_start_time+t+T_MPC].values
    price_buy = price_data[sim_start_time+t:sim_start_time+t+T_MPC].values
    month_slices = get_month_slices(baseline[sim_start_time+t:sim_start_time+t+T_MPC].index)
    return p_load, price_buy, month_slices
    
def implement(t, q_value, p_grid_value, prev_peak):
    energy_stored[t] = np.clip(q_value, 0, None)
    q_init = energy_stored[t]
    
    power[t] = p_grid_value
    peak[t] = np.max([power[t], prev_peak])
    
    if T - t > 1:
        if baseline.index[sim_start_time+t].month != baseline.index[sim_start_time+t+1].month:
            prev_peak = 0
        else:
            prev_peak = peak[t]
    return prev_peak, q_init
    
for t in tqdm.trange(T, desc="Simulating MPC"):
    # Shrinking horizon
    if T - t < T_MPC:
        T_MPC = T - t
    # Update forecasts
    p_load, price_buy, month_slices = make_forecasts(t, T_MPC)
    # Create problem and solve
    q_value, p_grid_value = solve_problem(T_MPC, p_load, price_buy, prev_peak, q_init, month_slices)
    # Implement MPC solution
    prev_peak, q_init = implement(t, q_value, p_grid_value, prev_peak)

# Compute cost
cost_usage = np.sum(price_data[sim_index] * power)
power_series = pd.Series(power, index=sim_index)
peak_series = power_series.resample('M').max().values
cost_peak = np.sum(peak_series * price_peak)
mpc_cost_baseline = cost_usage + cost_peak

# Suboptimality gap
absolute_gap = mpc_cost_baseline - prescient_cost
relative_gap = absolute_gap / mpc_cost_baseline
print(f"Absolute gap: {absolute_gap:.2f}")
print(f"Relative gap: {relative_gap * 100:.2f}%")

# Report results
print_costs(cost_usage, cost_peak)
plot_power_grid(sim_index, power, Q)
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

plot_state_of_charge(sim_index, energy_stored)
plt.gcf().set_size_inches(fig_width, 0.5*fig_height)

if save_figs:
    plt.savefig("figures/mpc_baseline_linear.pdf")